In [1]:
import torch
from torch import nn

In [2]:
import sys
sys.path.insert(0, '..')
from common import d2l

#### 1. 互相关运算

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])

In [4]:
d2l.corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

#### 2. 卷积层

In [5]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        """
        :param kernel_size: 卷积核形状
        """
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, X):
        return d2l.corr2d(X, self.weight) + self.bias

#### 3.图像中目标的边缘检测

In [6]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
# 卷积核
K = torch.tensor([[-1.0, 1.0]])

In [8]:
Y = d2l.corr2d(X, K)  # 检测出了纵向的边缘
Y

tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

In [9]:
d2l.corr2d(X.T, K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

#### 4. 学习卷积核

In [10]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

In [11]:
X = X.reshape(1, 1, 6, 8)
Y = Y.reshape(1, 1, 6, 7)

In [12]:
lr = 3e-2

In [13]:
for i in range(20):
    Y_hat = conv2d(X)
    loss = (Y - Y_hat) ** 2
    conv2d.zero_grad()
    loss.sum().backward()
    # 更新参数
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    print(f"第{i + 1}轮迭代，loss {loss.sum():f}")

第1轮迭代，loss 37.425594
第2轮迭代，loss 21.030256
第3轮迭代，loss 12.262462
第4轮迭代，loss 7.357726
第5轮迭代，loss 4.508140
第6轮迭代，loss 2.802959
第7轮迭代，loss 1.760204
第8轮迭代，loss 1.112731
第9轮迭代，loss 0.706496
第10轮迭代，loss 0.449842
第11轮迭代，loss 0.286951
第12轮迭代，loss 0.183260
第13轮迭代，loss 0.117127
第14轮迭代，loss 0.074896
第15轮迭代，loss 0.047907
第16轮迭代，loss 0.030650
第17轮迭代，loss 0.019611
第18轮迭代，loss 0.012549
第19轮迭代，loss 0.008031
第20轮迭代，loss 0.005139


In [14]:
conv2d.weight.data

tensor([[[[-0.9925,  1.0073]]]])